In [1]:
import pystac_client
from shapely.geometry import mapping

from rastervision.pipeline.file_system.utils import json_to_file
from rastervision.core.box import Box
from rastervision.core.data import XarraySourceConfig, STACItemConfig

In [2]:
bbox = Box(ymin=48.8155755, xmin=2.224122, ymax=48.902156, xmax=2.4697602)
bbox_geometry = mapping(bbox.to_shapely().oriented_envelope)

In [7]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)

items = catalog.search(
    intersects=bbox_geometry,
    collections=["sentinel-2-l2a"],
    datetime="2023-06-01/2023-06-20",
).get_all_items()

In [10]:
item = items[0]
json_to_file(item.to_dict(), 'xarray_source_config/item.json')

In [3]:
raster_source_config = XarraySourceConfig(
    stac=STACItemConfig(
        uri='xarray_source_config/item.json', 
        assets=[
        'coastal', # B01
        'blue', # B02
        'green', # B03
        # 'red', # B04
        # 'rededge1', # B05
        # 'rededge2', # B06
        # 'rededge3', # B07
        # 'nir', # B08
        # 'nir08', # B8A
        # 'nir09', # B09
        # 'swir16', # B11
        # 'swir22', # B12
    ]),
    bbox_map_coords=tuple(bbox),
)

In [5]:
rs = raster_source_config.build()

In [3]:
from os.path import join

from rastervision.pipeline.file_system.utils import json_to_file
from rastervision.core.data import (
    RasterioSourceConfig, SceneConfig, SemanticSegmentationLabelStoreConfig)
from rastervision.core.predictor import Predictor2

In [11]:
label_store_config = SemanticSegmentationLabelStoreConfig(
    uri='./xarray_source_config/pred',
)

In [19]:
scene_config = SceneConfig(
    id='test_scene',
    raster_source=raster_source_config,
    label_store=label_store_config
)

In [4]:
scene_config_uri = join('./xarray_source_config/pred', 'scene-config.json')
json_to_file(scene_config.dict(), scene_config_uri)

In [5]:
model_bundle_uri = 'https://s3.amazonaws.com/azavea-research-public-data/raster-vision/examples/model-zoo-0.21/isprs-potsdam-ss/model-bundle.zip'

In [6]:
predictor = Predictor2(model_bundle_uri)

2023-11-05 18:33:08:rastervision.pipeline.file_system.utils: INFO - Using cached file /opt/data/tmp/cache/http/s3.amazonaws.com/azavea-research-public-data/raster-vision/examples/model-zoo-0.21/isprs-potsdam-ss/model-bundle.zip.
2023-11-05 18:33:09:rastervision.pytorch_learner.learner: INFO - Loading learner from bundle /opt/data/tmp/tmpajpm8oa2/bundle/model-bundle.zip.
2023-11-05 18:33:09:rastervision.pytorch_learner.learner: INFO - Unzipping model-bundle to /opt/data/tmp/tmpajpm8oa2/model-bundle
2023-11-05 18:33:09:rastervision.pytorch_learner.learner: INFO - Using model definition found in bundle: /opt/data/tmp/tmpajpm8oa2/model-bundle/modules/fpn
2023-11-05 18:33:09:rastervision.pytorch_learner.learner: INFO - Local output dir: /opt/data/tmp/tmpajpm8oa2/s3/raster-vision/examples/0.21/output/isprs-potsdam-ss/train
2023-11-05 18:33:09:rastervision.pytorch_learner.learner: INFO - Remote output dir: s3://raster-vision/examples/0.21/output/isprs-potsdam-ss/train
2023-11-05 18:33:09:rast

In [7]:
%env RASTERVISION_PREDICT_NUM_WORKERS=0

env: RASTERVISION_PREDICT_NUM_WORKERS=2


In [21]:
predictor.predict(scene_config_uri)

Making predictions on test_scene:   0%|          | 0/18 [00:00<?, ?it/s]

Saving pixel labels:   0%|          | 0/4 [00:00<?, ?it/s]

CLI:

In [17]:
!rastervision predict_scene {model_bundle_uri} {scene_config_uri}

2023-11-05 18:49:33:rastervision.pipeline.file_system.utils: INFO - Using cached file /opt/data/tmp/cache/http/s3.amazonaws.com/azavea-research-public-data/raster-vision/examples/model-zoo-0.21/isprs-potsdam-ss/model-bundle.zip.
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Loading learner from bundle /opt/data/tmp/tmpsm6j2_e7/bundle/model-bundle.zip.
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Unzipping model-bundle to /opt/data/tmp/tmpsm6j2_e7/model-bundle
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Using model definition found in bundle: /opt/data/tmp/tmpsm6j2_e7/model-bundle/modules/fpn
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Local output dir: /opt/data/tmp/tmpsm6j2_e7/s3/raster-vision/examples/0.21/output/isprs-potsdam-ss/train
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Remote output dir: s3://raster-vision/examples/0.21/output/isprs-potsdam-ss/train
2023-11-05 18:49:34:rast

In [17]:
!rastervision predict_scene 'eurosat/train/' {scene_config_uri}

2023-11-05 18:49:33:rastervision.pipeline.file_system.utils: INFO - Using cached file /opt/data/tmp/cache/http/s3.amazonaws.com/azavea-research-public-data/raster-vision/examples/model-zoo-0.21/isprs-potsdam-ss/model-bundle.zip.
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Loading learner from bundle /opt/data/tmp/tmpsm6j2_e7/bundle/model-bundle.zip.
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Unzipping model-bundle to /opt/data/tmp/tmpsm6j2_e7/model-bundle
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Using model definition found in bundle: /opt/data/tmp/tmpsm6j2_e7/model-bundle/modules/fpn
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Local output dir: /opt/data/tmp/tmpsm6j2_e7/s3/raster-vision/examples/0.21/output/isprs-potsdam-ss/train
2023-11-05 18:49:34:rastervision.pytorch_learner.learner: INFO - Remote output dir: s3://raster-vision/examples/0.21/output/isprs-potsdam-ss/train
2023-11-05 18:49:34:rast